In [12]:
from pyspark.sql import SparkSession, dataframe
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql import HiveContext
from pyspark.sql.functions import *
from pyspark.sql import functions as f
from pyspark.sql.window import Window
import importlib
import json
from typing import Dict, Tuple
import argparse
import pandas as pd



In [13]:
#transferindo as tabelas para o spark e transformando em dataframe 

# Lista com nomes tabelas
tabelas = ["clientes", "divisao", "endereco", "regiao", "vendas"]

# Dicionário para armazenar os DataFrames
df_pandas = {}

# Loop para ler as tabelas e armazenar os DataFrames no dicionário
for tabela in tabelas:
    query = "SELECT * FROM desafio_curso." + tabela
    df = spark.sql(query)
    
    #transforma em df_pandas ja removendo o index e resetando
    df_pandas[tabela] = df.toPandas().drop(index=0).reset_index(drop=True)

In [14]:
df_pandas['vendas']

,actual_delivery_date,customerkey,datekey,discount_amount,invoice_date,invoice_number,item_class,item_number,item,line_number,...,order_number,promised_delivery_date,sales_amount,sales_amount_based_on_list_price,sales_cost_amount,sales_margin_amount,sales_price,sales_quantity,sales_rep,um
0,28/04/2019,10000481,28/04/2018,"-237,91",30/04/2018,100012,,,Urban Large Eggs,2000,...,200015,28/04/2019,"237,91",0,0,"237,91","237,91",1,184,EA
1,12/07/2019,10002220,12/07/2018,"368,79",14/07/2018,100233,P01,20910,Moms Sliced Turkey,1000,...,200245,12/07/2019,"456,17","824,96",0,"456,17","456,17",1,127,EA
2,14/10/2019,10002220,15/10/2018,"109,73",17/10/2018,116165,P01,38076,Cutting Edge Foot-Long Hot Dogs,1000,...,213157,14/10/2019,"438,93","548,66",0,"438,93","438,93",1,127,EA
3,01/06/2019,10002489,01/06/2018,"-211,75",03/06/2018,100096,,,Kiwi Lox,1000,...,200107,01/06/2019,"211,75",0,0,"211,75","211,75",1,160,EA
4,26/05/2019,10004516,25/05/2018,"96627,94",27/05/2018,103341,P01,60776,High Top Sweet Onion,1000,...,203785,26/05/2019,"89248,66","185876,6",0,"89248,66","196,1509011",455,124,SE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65530,,,,,,,,,,,...,,,,,,,,,,
65531,,,,,,,,,,,...,,,,,,,,,,
65532,,,,,,,,,,,...,,,,,,,,,,
65533,,,,,,,,,,,...,,,,,,,,,,


In [15]:
### REMOVENDO REPETIDOS NAS TABELAS
df_pandas['clientes']  = df_pandas['clientes'].drop_duplicates(subset='customerkey')
df_pandas['endereco']  = df_pandas['endereco'] .drop_duplicates(subset='address_number')
df_pandas['vendas']  = df_pandas['vendas'] .drop_duplicates(subset='order_number')

In [16]:
#Listando as tabelas para analisar e visulizar melhor

In [17]:
df_pandas['clientes'].head()

,address_number,business_family,business_unit,customer,customerkey,customer_type,division,line_of_business,phone,region_code,regional_sales_mgr,search_type
0,10000000,R3,1,City Supermarket,10000000,G2,2,,816-455-8733,4,S16,C
1,10000453,R3,1,A Supermarket,10000453,G1,1,,816-455-8733,5,S19,C
2,10000455,R3,1,Caribian Supermarket,10000455,G2,2,,816-455-8733,1,S16,C
3,10000456,R1,1,A&B Shop,10000456,G3,1,,816-455-8733,0,S2,C
4,10000457,O2,1,A&G Shop,10000457,G1,1,,816-455-8733,5,S1,C


In [18]:
# criando filtros de tabela clientes
filtro = {}
filtro['clientes'] = ['business_family','line_of_business','regional_sales_mgr','search_type']

In [19]:
df_pandas['divisao']

,division,division_name
0,1,International
1,2,Domestic


In [20]:
df_pandas['endereco'].head()

,address_number,city,country,customer_address_1,customer_address_2,customer_address_3,customer_address_4,state,zip_code
0,10000000,Akron,US,PO Box 6258,,,,OH,44312
1,10000453,,UK,,,,,,
2,10000455,Huntington Beach,US,7392 Count Circle,,,,CA,92647
3,10000456,Edmonton,CA,8151 Wagner Road,,,,AB,T6E 4N6
4,10000458,Saginaw,US,PO Box 840,,,,MI,48606


In [21]:
df_pandas['regiao']

,region_code,region_name
0,0,Canada
1,1,Western
2,2,Southern
3,3,Northeast
4,4,Central
5,5,International


In [22]:
df_pandas['vendas'].head()

,actual_delivery_date,customerkey,datekey,discount_amount,invoice_date,invoice_number,item_class,item_number,item,line_number,...,order_number,promised_delivery_date,sales_amount,sales_amount_based_on_list_price,sales_cost_amount,sales_margin_amount,sales_price,sales_quantity,sales_rep,um
0,28/04/2019,10000481,28/04/2018,"-237,91",30/04/2018,100012,,,Urban Large Eggs,2000,...,200015,28/04/2019,"237,91",0,0,"237,91","237,91",1,184,EA
1,12/07/2019,10002220,12/07/2018,"368,79",14/07/2018,100233,P01,20910,Moms Sliced Turkey,1000,...,200245,12/07/2019,"456,17","824,96",0,"456,17","456,17",1,127,EA
2,14/10/2019,10002220,15/10/2018,"109,73",17/10/2018,116165,P01,38076,Cutting Edge Foot-Long Hot Dogs,1000,...,213157,14/10/2019,"438,93","548,66",0,"438,93","438,93",1,127,EA
3,01/06/2019,10002489,01/06/2018,"-211,75",03/06/2018,100096,,,Kiwi Lox,1000,...,200107,01/06/2019,"211,75",0,0,"211,75","211,75",1,160,EA
4,26/05/2019,10004516,25/05/2018,"96627,94",27/05/2018,103341,P01,60776,High Top Sweet Onion,1000,...,203785,26/05/2019,"89248,66","185876,6",0,"89248,66","196,1509011",455,124,SE


In [23]:
# Tratando valores vazios do tipo String pra cada coluna

In [24]:
##### ENDERECO

In [25]:
#identificando e preenchendo valores vazios de endereço

#selecionando colunas para verificação
colunas= ['city','country','customer_address_1','customer_address_2','customer_address_3','customer_address_4','state','zip_code']

df = df_pandas['endereco']
# Verificar valores vazios na coluna
for coluna in colunas:
    #remove os espaços em branco e troca pra "nao informado"
    df[coluna] = df[coluna].str.strip().replace('','Não informado')
  
    
df_pandas['endereco'].head()

,address_number,city,country,customer_address_1,customer_address_2,customer_address_3,customer_address_4,state,zip_code
0,10000000,Akron,US,PO Box 6258,Não informado,Não informado,Não informado,OH,44312
1,10000453,Não informado,UK,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
2,10000455,Huntington Beach,US,7392 Count Circle,Não informado,Não informado,Não informado,CA,92647
3,10000456,Edmonton,CA,8151 Wagner Road,Não informado,Não informado,Não informado,AB,T6E 4N6
4,10000458,Saginaw,US,PO Box 840,Não informado,Não informado,Não informado,MI,48606


In [26]:
##### VENDAS

In [27]:
#identificando e preenchendo valores vazios de vendas

#selecionando colunas para verificação
colunas= ['item','item_class','um']

df = df_pandas['vendas']
# Verificar valores vazios na coluna
for coluna in colunas:
    #remove os espaços em branco e troca pra "nao informado"
    df[coluna] = df[coluna].str.strip().replace('','Não informado')
  
    
df_pandas['vendas'].head()

/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,actual_delivery_date,customerkey,datekey,discount_amount,invoice_date,invoice_number,item_class,item_number,item,line_number,...,order_number,promised_delivery_date,sales_amount,sales_amount_based_on_list_price,sales_cost_amount,sales_margin_amount,sales_price,sales_quantity,sales_rep,um
0,28/04/2019,10000481,28/04/2018,"-237,91",30/04/2018,100012,Não informado,,Urban Large Eggs,2000,...,200015,28/04/2019,"237,91",0,0,"237,91","237,91",1,184,EA
1,12/07/2019,10002220,12/07/2018,"368,79",14/07/2018,100233,P01,20910,Moms Sliced Turkey,1000,...,200245,12/07/2019,"456,17","824,96",0,"456,17","456,17",1,127,EA
2,14/10/2019,10002220,15/10/2018,"109,73",17/10/2018,116165,P01,38076,Cutting Edge Foot-Long Hot Dogs,1000,...,213157,14/10/2019,"438,93","548,66",0,"438,93","438,93",1,127,EA
3,01/06/2019,10002489,01/06/2018,"-211,75",03/06/2018,100096,Não informado,,Kiwi Lox,1000,...,200107,01/06/2019,"211,75",0,0,"211,75","211,75",1,160,EA
4,26/05/2019,10004516,25/05/2018,"96627,94",27/05/2018,103341,P01,60776,High Top Sweet Onion,1000,...,203785,26/05/2019,"89248,66","185876,6",0,"89248,66","196,1509011",455,124,SE


In [28]:
#### CLIENTES 
df_pandas['clientes'].head()

,address_number,business_family,business_unit,customer,customerkey,customer_type,division,line_of_business,phone,region_code,regional_sales_mgr,search_type
0,10000000,R3,1,City Supermarket,10000000,G2,2,,816-455-8733,4,S16,C
1,10000453,R3,1,A Supermarket,10000453,G1,1,,816-455-8733,5,S19,C
2,10000455,R3,1,Caribian Supermarket,10000455,G2,2,,816-455-8733,1,S16,C
3,10000456,R1,1,A&B Shop,10000456,G3,1,,816-455-8733,0,S2,C
4,10000457,O2,1,A&G Shop,10000457,G1,1,,816-455-8733,5,S1,C


In [29]:
#identificando e preenchendo valores vazios de clientes

#selecionando colunas para verificação
colunas= ['customer','customer_type','line_of_business','phone']

df = df_pandas['clientes']
# Verificar valores vazios na coluna
for coluna in colunas:
    #remove os espaços em branco e troca pra "nao informado"
    df[coluna] = df[coluna].str.strip().replace('','Não informado')
  
    
df_pandas['clientes'].head()

,address_number,business_family,business_unit,customer,customerkey,customer_type,division,line_of_business,phone,region_code,regional_sales_mgr,search_type
0,10000000,R3,1,City Supermarket,10000000,G2,2,Não informado,816-455-8733,4,S16,C
1,10000453,R3,1,A Supermarket,10000453,G1,1,Não informado,816-455-8733,5,S19,C
2,10000455,R3,1,Caribian Supermarket,10000455,G2,2,Não informado,816-455-8733,1,S16,C
3,10000456,R1,1,A&B Shop,10000456,G3,1,Não informado,816-455-8733,0,S2,C
4,10000457,O2,1,A&G Shop,10000457,G1,1,Não informado,816-455-8733,5,S1,C


In [30]:
# Tratando valores vazios do tipo decimais ou inteiros  pra cada coluna

In [31]:
#### ENDERECO

In [32]:
#identificando e preenchendo valores vazios de endereço

#selecionando colunas para verificação
colunas= ['address_number']

df = df_pandas['endereco']
# Verificar valores vazios na coluna
for coluna in colunas:
    #remove os espaços em branco e troca pra "nao informado"
    df[coluna] = df[coluna].str.strip().replace('','0')
  
    


In [33]:
##### VENDAS

In [34]:
#identificando e preenchendo valores vazios de endereço

colunas= ['customerkey','discount_amount','invoice_number','item_number','sales_amount','sales_amount_based_on_list_price','sales_price','sales_quantity']

df = df_pandas['vendas']
# Verificar valores vazios na coluna
for coluna in colunas:
    #remove os espaços em branco e troca pra "0"
    df[coluna] = df[coluna].str.strip().replace('','0')

        



/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [35]:
##### CLIENTES

df_pandas['clientes'].head()

,address_number,business_family,business_unit,customer,customerkey,customer_type,division,line_of_business,phone,region_code,regional_sales_mgr,search_type
0,10000000,R3,1,City Supermarket,10000000,G2,2,Não informado,816-455-8733,4,S16,C
1,10000453,R3,1,A Supermarket,10000453,G1,1,Não informado,816-455-8733,5,S19,C
2,10000455,R3,1,Caribian Supermarket,10000455,G2,2,Não informado,816-455-8733,1,S16,C
3,10000456,R1,1,A&B Shop,10000456,G3,1,Não informado,816-455-8733,0,S2,C
4,10000457,O2,1,A&G Shop,10000457,G1,1,Não informado,816-455-8733,5,S1,C


In [36]:
#identificando e preenchendo valores vazios de endereço

colunas= ['address_number','business_unit','customerkey','division']
df = df_pandas['clientes']
# Verificar valores vazios na coluna
for coluna in colunas:
    #remove os espaços em branco e troca pra "0"
    df[coluna] = df[coluna].str.strip().replace('','0')
  
df_pandas['vendas'].head()

,actual_delivery_date,customerkey,datekey,discount_amount,invoice_date,invoice_number,item_class,item_number,item,line_number,...,order_number,promised_delivery_date,sales_amount,sales_amount_based_on_list_price,sales_cost_amount,sales_margin_amount,sales_price,sales_quantity,sales_rep,um
0,28/04/2019,10000481,28/04/2018,"-237,91",30/04/2018,100012,Não informado,0,Urban Large Eggs,2000,...,200015,28/04/2019,"237,91",0,0,"237,91","237,91",1,184,EA
1,12/07/2019,10002220,12/07/2018,"368,79",14/07/2018,100233,P01,20910,Moms Sliced Turkey,1000,...,200245,12/07/2019,"456,17","824,96",0,"456,17","456,17",1,127,EA
2,14/10/2019,10002220,15/10/2018,"109,73",17/10/2018,116165,P01,38076,Cutting Edge Foot-Long Hot Dogs,1000,...,213157,14/10/2019,"438,93","548,66",0,"438,93","438,93",1,127,EA
3,01/06/2019,10002489,01/06/2018,"-211,75",03/06/2018,100096,Não informado,0,Kiwi Lox,1000,...,200107,01/06/2019,"211,75",0,0,"211,75","211,75",1,160,EA
4,26/05/2019,10004516,25/05/2018,"96627,94",27/05/2018,103341,P01,60776,High Top Sweet Onion,1000,...,203785,26/05/2019,"89248,66","185876,6",0,"89248,66","196,1509011",455,124,SE


In [37]:
## REMOVENDO COLUNAS 

In [38]:
### VENDAS

In [39]:
### nesse momento trabalharei com spark, apenas para utilizar os modos de manipulação mesmo 

In [40]:
### convertendo para spark 

In [41]:
df_spark = {}
df_panda ={}
for tabela in tabelas:
    df_panda = df_pandas[tabela] 
    # Converter DataFrame do pandas para DataFrame do Spark
    df_spark[tabela] = spark.createDataFrame(df_panda)

In [42]:
df_spark['clientes'].show(5)

+--------------+---------------+-------------+--------------------+-----------+-------------+--------+----------------+------------+-----------+------------------+-----------+
|address_number|business_family|business_unit|            customer|customerkey|customer_type|division|line_of_business|       phone|region_code|regional_sales_mgr|search_type|
+--------------+---------------+-------------+--------------------+-----------+-------------+--------+----------------+------------+-----------+------------------+-----------+
|      10000000|             R3|            1|    City Supermarket|   10000000|           G2|       2|   Não informado|816-455-8733|          4|               S16|          C|
|      10000453|             R3|            1|       A Supermarket|   10000453|           G1|       1|   Não informado|816-455-8733|          5|               S19|          C|
|      10000455|             R3|            1|Caribian Supermarket|   10000455|           G2|       2|   Não informado|8

In [43]:
### JUNTANDO COLUNAS entra tabelas

## CLIENTE E DIVISAO
df_cliente_divisao = df_spark['clientes'].join(df_spark['divisao'], on='division', how='inner')
df_cliente_divisao = df_spark['clientes'].join(df_spark['divisao'], on='division', how='inner')

## CLIENTE E REGIAO
df_cliente_stage = df_cliente_divisao.join(df_spark['regiao'], on='region_code', how='inner')


#CLIENTE E ENDERECO
df_cliente_stage = df_cliente_stage.join(df_spark['endereco'], on='address_number', how='inner')


In [44]:
### JUNTANDO TABELA DE VENDAS 

#note: antes de manipular mais a tabela, irei remover colunas e fazer contas

#criando um fitro
colunas_remover = ['actual_delivery_date','datekey','line_number','promised_delivery_date','sales_amount_based_on_list_price','sales_margin_amount','sales_rep','um']

#removendo as colunas
df_vendas_filtrada = df_spark['vendas'].drop(*colunas_remover)



In [45]:
### Criando o indicador de soma total das quantidades

# Converter a coluna 'sales_quantity' para o tipo numérico
df = df_vendas_filtrada.withColumn('sales_quantity', col('sales_quantity').cast('double'))

# Definir a janela de operação
window = Window.orderBy()

# Adicionar uma nova coluna 'total_sales' com a soma dos valores da coluna 'sales_quantity'
df = df.withColumn('total_sales', sum(col('sales_quantity')).over(window))

##repete o mesmos passoas para calcular o total do valor das vendas
df_aux = df.withColumn('sales_amount', col('sales_amount').cast('double'))
df = df_aux.withColumn('total_sales_price', sum(col('sales_amount')).over(window))

# Dividir a coluna "invoice_date"
df = df.withColumn("day", split(col("invoice_date"), "/")[0].cast("int"))
df = df.withColumn("month", split(col("invoice_date"), "/")[1].cast("int"))
df = df.withColumn("year", split(col("invoice_date"), "/")[2].cast("int"))



df_vendas_filtrada = df 


In [46]:
### JUNTANDO AS INFORMAÇÔES DE VENDA COM CLIENTE

df_stage = df_vendas_filtrada.join(df_cliente_stage, on='customerkey', how='inner')



In [47]:
### CRIANDO CHAVES PRAS TABELAS FINALS

In [48]:

#alterando o nome da coluna para ficar mais padronizada 

df_stage = df_stage.withColumnRenamed("division", "division_code")
df_stage = df_stage.withColumnRenamed("item", "item_name")
df_stage = df_stage.withColumnRenamed("customerkey", "customer_code")
df_stage = df_stage.withColumnRenamed("customer", "customer_name")

In [49]:

#DIMENSAO LOCALIDADE
df_stage = df_stage.withColumn("DW_LOCALIDADE", sha2(concat_ws("", df_stage.address_number,df_stage.state,df_stage.division_name, df_stage.division_code), 256))

#DIMENSAO TEMPO
df_stage = df_stage.withColumn("DW_TEMPO", sha2(concat_ws("", df_stage.invoice_date, df_stage.year,df_stage.month,df_stage.day), 256))

#DIMENSAO CLEINTES
df_stage = df_stage.withColumn("DW_CLIENTES", sha2(concat_ws("", df_stage.customer_name, df_stage.customer_code), 256))

In [50]:
df_stage.toPandas().head()

,customer_code,discount_amount,invoice_date,invoice_number,item_class,item_number,item_name,list_price,order_number,sales_amount,...,country,customer_address_1,customer_address_2,customer_address_3,customer_address_4,state,zip_code,DW_LOCALIDADE,DW_TEMPO,DW_CLIENTES
0,10000472,"363,99",21/01/2018,120547,P01,61752,Carlson Cheese Spread,"803,86",216973,NaN,...,US,8000 Haskell Avenue,Não informado,Não informado,Não informado,CA,91406,3cdae08986eadfe94f029586b07616595067d643c80d73...,fb1360981e26f9507bf264e232c8da6a882b8683a2cff6...,8310d327d8032b1386ac3820a7551a4a53b6b039303946...
1,10000472,"919,75",25/01/2018,121031,P01,27550,Imagine Popsicles,"1084,61",217830,NaN,...,US,8000 Haskell Avenue,Não informado,Não informado,Não informado,CA,91406,3cdae08986eadfe94f029586b07616595067d643c80d73...,c9508c04d6ce0c46dca5aefbaea1ca3a6d896c2e6fbb9e...,8310d327d8032b1386ac3820a7551a4a53b6b039303946...
2,10000472,"334,45",11/02/2018,122466,P01,28761,Ebony Fuji Apples,"157,76",218734,NaN,...,US,8000 Haskell Avenue,Não informado,Não informado,Não informado,CA,91406,3cdae08986eadfe94f029586b07616595067d643c80d73...,8d3a9c0fb2fd1f7a932f603dcaea2a7196b420f682925b...,8310d327d8032b1386ac3820a7551a4a53b6b039303946...
3,10000472,"334,45",22/02/2018,123476,P01,28761,Ebony Fuji Apples,"157,76",219480,NaN,...,US,8000 Haskell Avenue,Não informado,Não informado,Não informado,CA,91406,3cdae08986eadfe94f029586b07616595067d643c80d73...,68c20a0f7d29fb31c7ae0d9e470f6f48067cf074c9be88...,8310d327d8032b1386ac3820a7551a4a53b6b039303946...
4,10000472,"253,65",05/03/2018,124390,P01,25300,Fast Dried Apples,"199,41",220087,NaN,...,US,8000 Haskell Avenue,Não informado,Não informado,Não informado,CA,91406,3cdae08986eadfe94f029586b07616595067d643c80d73...,02db1a6d59c233ec1a61597f572d8c85d56ec49458cdc3...,8310d327d8032b1386ac3820a7551a4a53b6b039303946...


In [51]:
### CRIANDO DIMENSÔES E FATO

In [52]:
df_stage.createOrReplaceTempView('stage')

#Criando a dimensão Clientes
dim_clientes = spark.sql('''
    SELECT DISTINCT
        DW_CLIENTES,
        customer_code,
        customer_name     
    FROM stage    
''')


#Criando a dimensão TEMPO
dim_tempo = spark.sql('''
    SELECT DISTINCT
        DW_TEMPO,
        year,
        month,
        day,
        invoice_date
    FROM stage    
''')


#Criando a dimensão LOCALIDADE
dim_localidade = spark.sql('''
    SELECT DISTINCT
        DW_LOCALIDADE,
        address_number,
        state,
        division_name,
        division_code
    FROM stage    
''')



#FATO!!


#Criando a Fato VENDAS!
ft_vendas = spark.sql('''
    SELECT 
        DW_CLIENTES,
        DW_TEMPO,
        DW_LOCALIDADE,   
        total_sales,
        sales_amount,
        total_sales_price
    FROM stage
    group by 
        DW_CLIENTES, 
        DW_TEMPO,
        DW_LOCALIDADE,
        total_sales,
        sales_amount,
        total_sales_price
''')



In [53]:
### SALVANDO OS DADOS

In [54]:
# função para salvar os dados
def salvar_df(df, file):
    output = "/input/gold/" + file
    erase = "hdfs dfs -rm " + output + "/*"
    rename = "hdfs dfs -get /datalake/gold/"+file+"/part-* /input/gold/"+file+".csv"
    print(rename)
    
    
    df.coalesce(1).write\
        .format("csv")\
        .option("header", True)\
        .option("delimiter", ";")\
        .mode("overwrite")\
        .save("/datalake/gold/"+file+"/")

    os.system(erase)
    os.system(rename)

In [55]:
salvar_df(ft_vendas, 'ft_vendas')
salvar_df(dim_clientes, 'dim_cliente')
salvar_df(dim_tempo, 'dim_tempo')
salvar_df(dim_localidade, 'dim_localidade')

hdfs dfs -get /datalake/gold/ft_vendas/part-* /input/gold/ft_vendas.csv
hdfs dfs -get /datalake/gold/dim_cliente/part-* /input/gold/dim_cliente.csv
hdfs dfs -get /datalake/gold/dim_tempo/part-* /input/gold/dim_tempo.csv
hdfs dfs -get /datalake/gold/dim_localidade/part-* /input/gold/dim_localidade.csv
